In [1]:
 %matplotlib tk
import os
import sys
import numpy as np
import theano

from kusanagi.ghost import control
from kusanagi.ghost import regression
from kusanagi.shell import cartpole
from kusanagi.ghost.algorithms import pilco, mc_pilco
from kusanagi.ghost.optimizers import ScipyOptimizer, SGDOptimizer
from kusanagi.base import apply_controller, train_dynamics, ExperienceDataset
from kusanagi import utils
from functools import partial
from matplotlib import pyplot as plt

# np.random.seed(1337)
np.set_printoptions(linewidth=500)

In [2]:
def plot_rollout(rollout_fn, *args, **kwargs):
    fig = kwargs.get('fig')
    axarr = kwargs.get('axarr')
    loss, costs, trajectories = rollout_fn(*args)
    n_samples, T, dims = trajectories.shape

    if fig is None or axarr is None:
        fig, axarr = plt.subplots(dims, sharex=True)
    exp_states = np.array(exp.states)
    for d in range(dims):
        axarr[d].clear()
        st = trajectories[:, :, d]
        # plot predictive distribution
        for i in range(n_samples):
            axarr[d].plot(
                np.arange(T-1), st[i, :-1], color='steelblue', alpha=0.3)
        # for i in range(len(exp.states)):
        #    axarr[d].plot(
        #         np.arange(T-1), exp_states[i,1:,d],
        #         color='orange', alpha=0.3)
        # plot experience
        axarr[d].plot(
            np.arange(T-1), np.array(exp.states[-1])[1:H, d], color='red')
        axarr[d].plot(
            np.arange(T-1), st[:, :-1].mean(0), color='purple')
    plt.show(block=False)
    plt.waitforbuttonpress(0.1)

    return fig, axarr

In [3]:
use_bnn_dyn = True
use_bnn_pol = True

# setup output directory
utils.set_output_dir(os.path.join(utils.get_output_dir(), 'cartpole'))

params = cartpole.default_params()
n_rnd = 4                           # number of random initial trials
n_opt = 100                         # learning iterations
n_samples = 50                      # number of MC samples if bayesian nn
H = params['max_steps']
gamma = params['discount']
angle_dims = params['angle_dims']

# initial state distribution
p0 = params['state0_dist']
D = p0.mean.size

In [4]:
# init environment
env = cartpole.Cartpole(**params['plant'])

# init policy
pol = control.NNPolicy(p0.mean, **params['policy'])\
    if use_bnn_pol else control.RBFPolicy(**params['policy'])
randpol = control.RandPolicy(maxU=pol.maxU)

# init dynmodel
dyn = regression.BNN(**params['dynamics_model'])\
    if use_bnn_dyn else regression.SSGP_UI(**params['dynamics_model'])

# init cost model
cost = partial(cartpole.cartpole_loss, **params['cost'])

# create experience dataset
exp = ExperienceDataset()

# init policy optimizer
if use_bnn_dyn:
    params['optimizer']['min_method'] = 'adam'
    params['optimizer']['max_evals'] = 1000
    polopt = SGDOptimizer(**params['optimizer'])
else:
    polopt = ScipyOptimizer(**params['optimizer'])

<class 'kusanagi.ghost.control.NNPolicy.NNPolicy'> True
[2017-08-21 18:41:32.638333] Experience > Initialising new experience dataset


In [5]:
# callback executed after every call to env.step
def step_cb(state, action, cost, info):
    exp.add_sample(state, action, cost, info)
    env.render()

def polopt_cb(*args, **kwargs):
    if hasattr(dyn, 'update'):
        dyn.update()
    if hasattr(pol, 'update'):
        pol.update()
        
    loss, dloss = args[:2]
    grad_norms = [np.sqrt((d.__array__()**2).sum()) for d in dloss]
    if sum(grad_norms) >  10.0*len(grad_norms):
        print([np.sqrt((d.__array__()**2).sum()) for d in dloss])

# function to execute before applying policy
def gTrig(state):
    return utils.gTrig_np(state, angle_dims).flatten()

# during first n_rnd trials, apply randomized controls
for i in range(n_rnd):
    exp.new_episode()
    apply_controller(env, randpol, H,
                     preprocess=gTrig,
                     callback=step_cb)

[2017-08-21 18:41:32.733976] apply_controller > Starting run
[2017-08-21 18:41:32.734848] apply_controller > Running for 4.000000 seconds


/home/juancamilog/miniconda3/lib/python3.6/site-packages/matplotlib/backend_bases.py:2453: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


[2017-08-21 18:41:35.035278] apply_controller > Done. Stopping robot. Value of run [39.664642]
[2017-08-21 18:41:35.036112] Cartpole > Stopping robot
[2017-08-21 18:41:35.036848] apply_controller > Starting run
[2017-08-21 18:41:35.037509] apply_controller > Running for 4.000000 seconds
[2017-08-21 18:41:37.029974] apply_controller > Done. Stopping robot. Value of run [39.989006]
[2017-08-21 18:41:37.030607] Cartpole > Stopping robot
[2017-08-21 18:41:37.031286] apply_controller > Starting run
[2017-08-21 18:41:37.032009] apply_controller > Running for 4.000000 seconds
[2017-08-21 18:41:39.029950] apply_controller > Done. Stopping robot. Value of run [39.978142]
[2017-08-21 18:41:39.030814] Cartpole > Stopping robot
[2017-08-21 18:41:39.031749] apply_controller > Starting run
[2017-08-21 18:41:39.032449] apply_controller > Running for 4.000000 seconds
[2017-08-21 18:41:41.048940] apply_controller > Done. Stopping robot. Value of run [39.996483]
[2017-08-21 18:41:41.050321] Cartpole > S

In [ ]:
# PILCO loop
rollout_fn = None
fig, axarr = None, None
for i in range(n_opt):
    total_exp = sum([len(st) for st in exp.states])
    msg = '==== Iteration [%d], experience: [%d steps] ===='
    utils.print_with_stamp(msg % (i+1, total_exp))

    # train dynamics model
    train_dynamics(dyn, exp, angle_dims=angle_dims)

    # initial state distribution
    x0 = np.array([st[0] for st in exp.states])
    m0 = x0.mean(0)
    S0 = np.cov(x0, rowvar=False, ddof=1) +\
        1e-4*np.eye(x0.shape[1]) if len(x0) > 2 else p0.cov

    if fig is not None:
        # plot rollout
        fig, axarr = plot_rollout(
            rollout_fn, m0, S0, H, gamma, fig=fig, axarr=axarr)

    # train policy
    if polopt.loss_fn is None or dyn.should_recompile:
        loss_kwargs = {}
        obj_kwargs = {}
        extra_inps = []
        if use_bnn_dyn:
            # init learning rate parameter
            lr = theano.tensor.scalar('lr')
            extra_inps += [lr]

            # parameters for building loss function
            loss_kwargs['n_samples'] = n_samples
            loss_kwargs['resample_particles'] = True
            obj_kwargs['learning_rate'] = lr
            obj_kwargs['clip'] = 10.0
            learner = mc_pilco
        else:
            learner = pilco

        # build loss function
        loss, inps, updts = learner.get_loss(
            pol, dyn, cost, D, angle_dims, **loss_kwargs)
        inps += extra_inps

        # set objective of policy optimizer
        polopt.set_objective(loss, pol.get_params(symbolic=True),
                             inps, updts, **obj_kwargs)

        # build rollout function for plotting
        if rollout_fn is None:
            loss_kwargs['resample_particles'] = False
            rollout_fn = learner.build_rollout(
                pol, dyn, cost, D, angle_dims, **loss_kwargs)

    polopt_args = [m0, S0, H, gamma]
    if use_bnn_dyn:
        polopt_args.append(1e-3)
    polopt.minimize(*polopt_args,
                    callback=polopt_cb)

    # apply controller
    exp.new_episode(policy_params=pol.get_params())
    apply_controller(env, pol, H,
                     preprocess=gTrig, callback=step_cb)

    # plot rollout
    fig, axarr = plot_rollout(
        rollout_fn, m0, S0, H, gamma, fig=fig, axarr=axarr)
input('Finished training')

[2017-08-21 18:41:41.239804] ==== Iteration [1], experience: [160 steps] ====
[2017-08-21 18:41:41.241125] train_dynamics > Training dynamics model
[2017-08-21 18:41:41.246082] train_dynamics > Dataset size:: Inputs: [ (156, 6) ], Targets: [ (156, 4) ]  
[2017-08-21 18:41:41.247390] BNN > Building network
InputLayer {'shape': (None, 6), 'name': 'BNN_input'}
DenseLayer {'num_units': 400, 'nonlinearity': <lasagne.nonlinearities.LeakyRectify object at 0x7f12f2966048>, 'W': <lasagne.init.Orthogonal object at 0x7f12f25d2160>, 'b': <lasagne.init.Constant object at 0x7f12f25d21d0>, 'name': 'BNN_fc0'}
DropoutLayer {'p': 0.05, 'rescale': False, 'name': 'BNN_drop0', 'n_samples': array(25, dtype=int32)}
DenseLayer {'num_units': 400, 'nonlinearity': <lasagne.nonlinearities.LeakyRectify object at 0x7f12f2966048>, 'W': <lasagne.init.Orthogonal object at 0x7f12f25d2160>, 'b': <lasagne.init.Constant object at 0x7f12f25d21d0>, 'name': 'BNN_fc1'}
DropoutLayer {'p': 0.05, 'rescale': False, 'name': 'BNN_d

/home/juancamilog/miniconda3/lib/python3.6/site-packages/matplotlib/backend_bases.py:2453: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


[2017-08-21 18:44:45.838853] apply_controller > Done. Stopping robot. Value of run [27.532969]
[2017-08-21 18:44:45.839659] Cartpole > Stopping robot
[2017-08-21 18:44:46.817714] ==== Iteration [2], experience: [200 steps] ====
[2017-08-21 18:44:46.818514] train_dynamics > Training dynamics model
[2017-08-21 18:44:46.820666] train_dynamics > Dataset size:: Inputs: [ (195, 6) ], Targets: [ (195, 4) ]  
[2017-08-21 18:44:46.822176] BNN_opt > Optimizing parameters via mini batches
[2017-08-21 18:44:46.838420] BNN_opt > Initial loss [2674.833251953125]
[2017-08-21 18:44:57.053801] BNN_opt > Curr loss: -9.706353E+02 [941: -9.752516E+02], n_evals: 1000, Avg. time per updt: 0.009239
[2017-08-21 18:44:57.072239] BNN_opt > Done training. New loss [-988.983337] iter: [941]
[2017-08-21 18:44:57.073032] train_dynamics > Done training dynamics model
[2017-08-21 18:44:57.808425] SGDOptimizer > Optimizing parameters
[2017-08-21 18:44:57.997725] SGDOptimizer > Initial loss [0.88953697681427]
[2017-08-